# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Couchard Darius - Parent Paul - Donne Stefano

## Pump it Up: Data Mining the Water Table
####  April 29, 2021


# 1) Data Pre-Processing




In [49]:
training_set<-read.csv("../Data/TrainingSet/4910797b-ee55-40a7-8668-10efd5c1b960.csv",header=TRUE) # loads the training set csv file (it's magic)
dim(training_set) # dimension of the set 
names(training_set) # names of the variables
#training_set


[1] 59400    40

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group"

## How to enhance the data set :
First, it's needed to remove empty values in the table: each NaN or empty cell has to be removed or replaced, it is decided for now to map Nan <- 0.

Then, modifications have to be made depending on the nature of the data :
* If a column (variable) consists in continous numerical values : standardization is applied such has obtaining a new column with a mean value of 0 and a standard deviation of 1 (**gps_height**)
* If a column is made of ordinal categorical variable (hierarchy between categories) : map each string to a numerical value (**water_quality**)
* In case of nominal categorical variable : apply one hot encoding -> create new column (with binary values) for each category (**source_type**)
<br/>

But some other cases have to be assesed :
* the name of the water point (**wpt_name**) isn't relevant to use as it is, as every water point has a unique or no name -> so either drop this column or map it as 1 : has name , 0 : no name
* the **funder** or **installer** variable cannot be addressed with one hot encoding as there is 1900 different funders, many of them having only 1 installation. So two solutions exists : drop the column (loss of information) or create new categories for funders (eg: number of installation per funder -> 1 , 1-10, 10+)
* ...
* todo : address variable redundancy (eg: **source** vs **source_type**)

## Methods are implement to apply these changes :

In [29]:
# METHOD TO REASSIGN EMPTY VALUES
NaN_handler <- function(column_name) { # input : column_name (name of the variable)
    training_set[training_set[,column_name] == "", ] <- 0 # select row where column element is empty string "" and assign to 0
    training_set[is.na(training_set[,column_name])] <- 0 # select row where column element is NaN and assign to 0
    return(training_set)
}
training_set<-NaN_handler("funder") # TEST

In [48]:
# STANDARDIZATION METHOD FOR VARIABLES WITH CONTINUOUS NUMERICAL VALUES
Standardization <- function(column_name){ # input : column_name (name of the variable)
    mean_col = mean(training_set[,column_name], na.rm = TRUE) # mean of the variable
    sd_col = sd(training_set[,column_name], na.rm = TRUE) # standard deviation of the variable
    training_set[,column_name]<-(training_set[,column_name]-mean_col)/sd_col # apply the transformation
    # now for the whole column : mean = 0 and sd = 1
    return(training_set)
}
training_set<-NaN_handler("gps_height") # TEST 
training_set<-Standardization("gps_height") # TEST
# also, amount_tsh, longitude, latitude, ...

In [3]:
require(tidyr)
require(dplyr)
# HANDLING OF NOMINAL CATEGORICAL VARIABLES
# before using : change notebook IOPub data rate limit with Jupyter  notebook --NotebookApp.iopub_data_rate_limit=jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
Nom_cat_handle <- function(column_name){
    training_set %>% mutate(value = 1)  %>% spread(column_name, value,  fill = 0 )
    return(training_set)
}
training_set<-Nom_cat_handle("source_type") # TEST

# THERE IS 1897 different funders ! many funding only 1 installation : it's needed to categorize them, no one hot encoding

id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,source_class,waterpoint_type,waterpoint_type_group,borehole,dam,other,rainwater harvesting,river/lake,shallow well,spring
<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>,<int>,...,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
69572,6000,2011-03-14,Roman,1390,Roman,34.93809,-9.85632177,none,0,...,groundwater,communal standpipe,communal standpipe,0,0,0,0,0,0,1
8776,0,2013-03-06,Grumeti,1399,GRUMETI,34.69877,-2.14746569,Zahanati,0,...,surface,communal standpipe,communal standpipe,0,0,0,1,0,0,0
34310,25,2013-02-25,Lottery Club,686,World vision,37.46066,-3.82132853,Kwa Mahundi,0,...,surface,communal standpipe multiple,communal standpipe,0,1,0,0,0,0,0
67743,0,2013-01-28,Unicef,263,UNICEF,38.48616,-11.15529772,Zahanati Ya Nanyumbu,0,...,groundwater,communal standpipe multiple,communal standpipe,1,0,0,0,0,0,0
19728,0,2011-07-13,Action In A,0,Artisan,31.13085,-1.82535885,Shuleni,0,...,surface,communal standpipe,communal standpipe,0,0,0,1,0,0,0
9944,20,2011-03-13,Mkinga Distric Coun,0,DWE,39.17280,-4.76558728,Tajiri,0,...,unknown,communal standpipe multiple,communal standpipe,0,0,1,0,0,0,0
19816,0,2012-10-01,Dwsp,0,DWSP,33.36241,-3.76636472,Kwa Ngomho,0,...,groundwater,hand pump,hand pump,1,0,0,0,0,0,0
54551,0,2012-10-09,Rwssp,0,DWE,32.62062,-4.22619802,Tushirikiane,0,...,groundwater,hand pump,hand pump,0,0,0,0,0,1,0
53934,0,2012-11-03,Wateraid,0,Water Aid,32.71110,-5.14671181,Kwa Ramadhan Musa,0,...,groundwater,hand pump,hand pump,1,0,0,0,0,0,0


In [50]:
library("writexl")
funder_occurency<-as.data.frame(table(training_set[,"funder"])) # data_frame containing the number of occurences of each funder
funder_occurency<-arrange(funder_occurency,Freq) # sorted in ascending frequency order
write_xlsx(funder_occurency,"../Data/PreProcess/funder_occ.csv") # stores occurences for later test_set pre-processing
funder_occurency[as.integer(nrow(funder_occurency)/3),2] # thresh 1 
funder_occurency[as.integer(2*nrow(funder_occurency)/3),2] # thresh 2
# the two thresholds split the data frame in 3 equal parts : (the number of 3 is arbitrary)
# - funders having opened 1 water pump
# - funders having opened 2 or 3 water pumps
# - funders having opened more than 3 water pumps
# The funder column can now be transformed, where every funder is now assigned to an ordinal categorical variable (1,2 or 3)
# 0 is already assigned by default the the rows without funder names

for(row in 1: nrow(training_set)){
    val = funder_occurency[which(funder_occurency[,1] == training_set[row,"funder"]),2]
    if(val>3){
        training_set[row,"funder"]<-3
    }
    else if (val>1){
        training_set[row,"funder"]<-2
    }
    else if (val >0){
        training_set[row,"funder"]<-1
    }
    else{
        training_set[row,"funder"]<-0
    }
    # reassign each funder value to its new category
}

# this process can be done for other very large nominal categorical variables

[1] 1

[1] 3

[1] "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
   [19] "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3"
   [37] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
   [55] "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3"
   [73] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
   [91] "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [109] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [127] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [145] "3" "3" "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [163] "3" "3" "3" "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [181] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [199] "3" "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3"
  [217] "3" "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3"
  [235] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [253] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [271] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [289] "3" "3" "3" "3" "3" "3" "3" "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [307] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [325] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [343] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "2"
  [361] "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [379] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [397] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [415] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3"
  [433] "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [451] "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [469] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [487] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [505] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [523] "3" "3" "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [541] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [559] "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [577] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [595] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [613] "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [631] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [649] "3" "3" "3" "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [667] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [685] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [703] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [721] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [739] "2" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [757] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [775] "2" "3" "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3" "3" "3" "3" "3" "3"
  [793] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "1" "3" "3"
  [811] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [829] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "2" "3" "3" "3" "1"
  [847] "2" "3" "3" "3" "3" "3" "1" "3" "3" "1" "3" "3" "3" "3" "1" "3" "1" "3"
  [865] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3"
  [883] "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "3" "2" "3" "3" "3"
  [9

In [7]:
# TEST : remap wpt_name with 0: no name, 1: has name
training_set[training_set[,"wpt_name"] == "none", ] <- 0
training_set[training_set[,"wpt_name"] != 0, ] <- 1

In [11]:
# TEST
training_set<-training_set[,!(names(training_set) %in% "wpt_name")] # drop the desired column